In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing

data = pd.read_csv("../input/data.csv")

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
def test_it_GBC(data_test):
    clf = GradientBoostingClassifier()
    return cross_val_score(clf, data_test.drop('shot_made_flag', 1), data_test.shot_made_flag,
                           scoring='log_loss', cv=10)

In [ ]:
# define the sort & enumeration function
def sort_encode(df, field):
    ct = pd.crosstab(df.shot_made_flag, df[field]).apply(lambda x:x/x.sum(), axis=0)
    temp = list(zip(ct.values[1, :], ct.columns))
    temp.sort(reverse=True)
    new_map = {}
    for index, (acc, old_number) in enumerate(temp):
        new_map[old_number] = index
    new_field = field + '_sort_enumerated'
    df[new_field] = df[field].map(new_map)
    return new_field

In [ ]:
sort_encode(data, 'action_type')

In [ ]:
sort_encode(data, 'shot_zone_area')

In [ ]:
# time_remaining
data['time'] = data['minutes_remaining']*60 + data['seconds_remaining']
data['time_new'] = 1000
data.loc[data.time>5, ['time_new']] = 0
data.loc[data.time<=5, ['time_new']] = 1

In [ ]:
# Impute
mode = data.action_type_sort_enumerated.mode()[0]
data.action_type_sort_enumerated.fillna(mode, inplace=True)
mode1 = data.shot_zone_area_sort_enumerated.mode()[0]
data.shot_zone_area_sort_enumerated.fillna(mode1, inplace=True)

In [ ]:
data['away'] = data.matchup.str.contains('@')

In [ ]:
data_test = data.loc[~data.shot_made_flag.isnull(),\
                     ['action_type_sort_enumerated','shot_distance','time_new',\
                      'away','shot_made_flag']]

In [ ]:
test_it_GBC(data_test).mean()

In [ ]:
train = data.loc[~data.shot_made_flag.isnull(), ['action_type_sort_enumerated', \
               'shot_distance', 'time_new', 'away', 'shot_made_flag']]
test = data.loc[data.shot_made_flag.isnull(), ['action_type_sort_enumerated', \
               'shot_distance', 'time_new', 'away', 'shot_id']]

In [ ]:
clf = GradientBoostingClassifier()

# Train and predict
clf.fit(train.drop('shot_made_flag',axis=1), train.shot_made_flag)
predictions = clf.predict_proba(test.drop('shot_id', axis=1))

# convert to CSV
submission = pd.DataFrame({'shot_id': test.shot_id, 'shot_made_flag': predictions[:, 1]})
submission[['shot_id', 'shot_made_flag']].to_csv('submission_GBC.csv', index=False)